In [1]:
DATA_PATH = "/home/giovanni/pod/thesis/code/scripts-sources/W3Pi-selection-algorithm/data/"


with open(DATA_PATH + "myfile.txt", "w") as f:
    f.write("Hello World!")

In [3]:
def sub_unsigned(pattern, hi, lo):
    return (pattern >> lo) & ((2 << (hi-lo))-1);

def sub_signed(pattern, hi, lo):
    raw = sub_unsigned(pattern, hi, lo)
    return raw-2**(hi-lo+1) if pattern & (1<<hi) else raw

eta_L_word = 330514441
eta = (eta_L_word >> 14) & 0xFFF
eta


3789

In [11]:
list(range(2, 3))

[2]

In [9]:
a = [1]
for _ in a:
    print(_)

1
